In [36]:
import pandas as pd
from sklearn.pipeline import Pipeline, FunctionTransformer
import numpy as np 
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

In [9]:
df = pd.read_excel('azeri_news.xlsx')

In [10]:
df

,Category,Title,News_Article
0,Maraqlı,Naviforce Sport Saat 2016 ilə zövqlərin ahəngi,Naviforce Sport Saat 2016 Yapon Mexanizmi Yapo...
1,Maraqlı,"Sinir ,oynaq , sinir bel ağrılarına 3 gündə son !","ŞOK ! ŞOK ! ŞOK ! Xanımlar və bəylər , bel və ..."
2,Maraqlı,Dəyərindən qat-qat aşağı qiymətə Mənzil,Dəyərindən qat-qat Aşağı Qiymətə. Həzi Aslanov...
3,İdman,2024 və 2028-ci il olimpiadalarının keçiriləcə...,2028-ci il Yay Olimpiya və Paralimpiya Oyunla...
4,Dünya,Türkiyədə zəlzələ,Türkiyədə daha bir zəlzələ meydana gəlib. L...
...,...,...,...
49995,Siyasət,Əli Kərimli terrorçularla da əməkdaşlığa hazır...,"AXCP sədri ""Milli Şura""nın mitinqində qəsdən..."
49996,Siyasət,Elşən Musayev: “Bəlkə elə o terrorist Əli Kəri...,"""Ümumiyyətlə, Milli Şuraya xoş olan, hətta o..."
49997,Dünya,İstanbulda 52 mərtəbəli binada yanğın baş verib,Türkiyənin İstanbul şəhərinin Ataşehir rayon...
49998,Dünya,“Onlar cinayətkarlardır və bir-bir məhv edilir...,Rusiyanı tərk edərək İŞİD sıralarında döyüşə...


In [11]:
df.Category.unique()

array(['Maraqlı', 'İdman', 'Dünya', 'Siyasət', 'İqtisadiyyat',
       'Mədəniyyət'], dtype=object)

In [12]:


def clean_text(text):
    az_letters = set('əöüşçğıƏÖÜŞÇĞI')
    
    text = ''.join(
        char.lower() if (char.isalpha() and char.isascii()) or char in az_letters else ' ' 
        for char in text
    )
    
    return re.sub(r' +', ' ', text).strip()

In [13]:
df['text'] = df['Title'] + ' ' + df['News_Article']
df['text'] = df['text'].apply(clean_text)

In [14]:
az_stopwords = [
    "a", "ad", "əgər", "amma", "ancaq", "artıq", "asılı", "az", "bax", "belə", "beləliklə",
    "bəli", "bəzən", "bəzi", "bütün", "bu", "buna", "bunda", "bundan", "bunu", "bunun",
    "burada", "bura", "bəri", "cəmi", "çox", "çünki",
    "da", "də", "daha", "demək", "deyil", "deyə", "dərhal", "dünən",
    "elə", "edir", "edən", "edərək", "edəcək", "edilir", "edilən", "edib", "et", "etdi", "etmək",
    "ə", "əldə", "əhəmiyyət", "əks", "əksinə", "əlaqədar", "ən", "etibarilə",
    "faktiki", "görə", "habelə", "hansı", "harada", "hara", "harda", "hazırda", "hə", "həmin",
    "hər", "hərçənd", "hələ", "heç", "heçbir", "heçbiri", "heçdə", "heçmi", "həm", "həmçinin",
    "xeyli", "xeyr", "xüsusilə",
    "içində", "ilə", "indi", "isə",
    "kifayət", "kimi", "kim", "kimin", "kimə", "kimdən", "kimdə", "kimdənsə", "kimisə", "kiminsə",
    "lakin",
    "məhz", "mən", "mənim", "mənə", "məndə", "məndən", "məni",
    "sən", "sənin", "sənə", "səndə", "səndən", "səni",
    "o", "onun", "ona", "onda", "ondan", "onu",
    "biz", "bizim", "bizə", "bizdə", "bizdən", "bizi",
    "siz", "sizin", "sizə", "sizdə", "sizdən", "sizi",
    "onlar", "onların", "onlara", "onlarda", "onlardan", "onları",
    "məncə", "səncə", "fikrincə",
    "məsələn", "məs", "məsəl", "müəyyən", "mütləq",
    "nə", "nəhayət", "nəinki", "nə qədər", "nə üçün", "nə zaman", "nə vaxt",
    "nədir", "nədən", "nədə", "nəyə", "nəni", "nəsə",
    "necə", "neçənci", "neçə", "neçəsi", "neçəsinin",
    "olaraq", "olur", "oldu", "olmuş", "olacaq", "olan", "olduğu", "olduğunu", "olsa", "olsun",
    "ona görə", "ona gör",
    "özü", "öz", "özüm", "özün", "özümdən", "özündən", "özündən", "özünə", "özününkülər",
    "qədər", "qarşı", "qəti", "qismən",
    "sanki", "sonra", "sonradan", "son", "sonuncu", "sualtı",
    "şübhəsiz",
    "təki", "təkcə", "tutaq", "tam", "tamamilə",
    "u", "üçün", "üstəlik",
    "və", "və ya", "və yaxud", "yaxud", "ya", "ya da",
    "yenə", "yenə də", "yəni", "yaxşı", "yox", "yoxsa",
    "zaman", "zamanı", 'bəlkə'
]


In [15]:
def remove_stopwords(text, remove_stopwords=True):
      
    if remove_stopwords:
        stopwords_set = set(az_stopwords)
        
        words = text.split()
        filtered_words = [word for word in words if word not in stopwords_set and len(word) > 3]
        text = " ".join(filtered_words)
        return text
    return text

In [16]:
df['text'] = df['text'].apply(remove_stopwords)

In [17]:
df

,Category,Title,News_Article,text
0,Maraqlı,Naviforce Sport Saat 2016 ilə zövqlərin ahəngi,Naviforce Sport Saat 2016 Yapon Mexanizmi Yapo...,naviforce sport saat zövqlərin ahəngi naviforc...
1,Maraqlı,"Sinir ,oynaq , sinir bel ağrılarına 3 gündə son !","ŞOK ! ŞOK ! ŞOK ! Xanımlar və bəylər , bel və ...",sinir oynaq sinir ağrılarına gündə xanımlar bə...
2,Maraqlı,Dəyərindən qat-qat aşağı qiymətə Mənzil,Dəyərindən qat-qat Aşağı Qiymətə. Həzi Aslanov...,dəyərindən aşağı qiymətə mənzil dəyərindən aşa...
3,İdman,2024 və 2028-ci il olimpiadalarının keçiriləcə...,2028-ci il Yay Olimpiya və Paralimpiya Oyunla...,olimpiadalarının keçiriləcəyi şəhərlər müəyyən...
4,Dünya,Türkiyədə zəlzələ,Türkiyədə daha bir zəlzələ meydana gəlib. L...,türkiyədə zəlzələ türkiyədə zəlzələ meydana gə...
...,...,...,...,...
49995,Siyasət,Əli Kərimli terrorçularla da əməkdaşlığa hazır...,"AXCP sədri ""Milli Şura""nın mitinqində qəsdən...",kərimli terrorçularla əməkdaşlığa hazırdır axc...
49996,Siyasət,Elşən Musayev: “Bəlkə elə o terrorist Əli Kəri...,"""Ümumiyyətlə, Milli Şuraya xoş olan, hətta o...",elşən musayev terrorist kərimlinin adamıdır üm...
49997,Dünya,İstanbulda 52 mərtəbəli binada yanğın baş verib,Türkiyənin İstanbul şəhərinin Ataşehir rayon...,stanbulda mərtəbəli binada yanğın verib türkiy...
49998,Dünya,“Onlar cinayətkarlardır və bir-bir məhv edilir...,Rusiyanı tərk edərək İŞİD sıralarında döyüşə...,cinayətkarlardır məhv edilirlər peskov rusiyan...


In [19]:
X = df['text']
y = df['Category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 2000, random_state=42)

vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)

In [22]:
X_train_vectorized

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 4257805 stored elements and shape (48000, 216120)>

In [23]:
def swap_dict(dictionary):
    return {value: key for key, value in dictionary.items()}

In [24]:
new_vocab_dict = swap_dict(vectorizer.vocabulary_)

In [25]:
new_vocab_dict

{119431: 'paris',
 189488: 'yaxınlığında',
 11050: 'avtomobil',
 141371: 'restorana',
 202030: 'çırpılıb',
 187927: 'yaralı',
 53573: 'fransanın',
 210725: 'şəhərinin',
 94933: 'markalı',
 98788: 'minik',
 11056: 'avtomobili',
 98314: 'milli',
 168572: 'trend',
 74149: 'istinadən',
 20720: 'bildirir',
 62800: 'hadisə',
 113834: 'nəticəsində',
 191121: 'yaşlı',
 177947: 'uşaq',
 68257: 'həlak',
 115509: 'olub',
 113359: 'nəfər',
 186061: 'xəsarət',
 4281: 'alıb',
 106226: 'məlumata',
 214491: 'əsasən',
 2765: 'alanların',
 182938: 'vəziyyəti',
 14520: 'ağırdır',
 11065: 'avtomobilin',
 158615: 'sürücüsü',
 147695: 'saxlanılıb',
 131800: 'qidalar',
 79061: 'karlığın',
 129458: 'qarşısını',
 4433: 'alır',
 186667: 'yahi',
 191068: 'yaşlandıqca',
 104337: 'müxtəlif',
 159855: 'səbəblərdən',
 50232: 'eşitməmizdə',
 109443: 'narahatlıqlar',
 188049: 'yaranır',
 131806: 'qidalarla',
 50225: 'eşitmədə',
 187966: 'yarana',
 21407: 'biləcək',
 124814: 'problemlərin',
 3637: 'almaq',
 103174: 'mü

In [26]:
X_test_vectorized = vectorizer.transform(X_test)

In [27]:
X_test_vectorized 

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 173518 stored elements and shape (2000, 216120)>

In [34]:
def clean_text_array(X):
    """Apply clean_text to each element in the array"""
    return np.array([clean_text(text) for text in X])

def remove_stopwords_array(X):
    """Apply remove_stopwords to each element in the array"""
    return np.array([remove_stopwords(text) for text in X])
    
    
text_pipeline = Pipeline([
    ('cleaner', FunctionTransformer(clean_text_array)),
    ('stopword_remover', FunctionTransformer(remove_stopwords_array)),
    ('vectorizer', TfidfVectorizer())
])

X = df['Title'] + df['News_Article'] 
y = df['Category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 2000, random_state=42)

X_train_vectorized = text_pipeline.fit_transform(X_train)
X_test_vectorized  = text_pipeline.transform(X_test)

In [37]:
def explain_model(model,  X_train, X_test, y_train, y_test, features = None):
    print('Test Results:')
    y_pred = model.predict(X_test)
    print(classification_report(y_pred, y_test))
    print(confusion_matrix(y_pred, y_test))
    
    print('Train Results:')
    y_pred_tr = model.predict(X_train)
    print(classification_report(y_pred_tr, y_train))
    print(confusion_matrix(y_pred_tr, y_train)) 
    
    print()
    print()

In [31]:
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

C:\Users\ufml\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [32]:
explain_model(model, X_train_vectorized, X_test_vectorized, y_train, y_test)

Test Results:
              precision    recall  f1-score   support

       Dünya       0.84      0.84      0.84       569
     Maraqlı       0.81      0.82      0.82       620
  Mədəniyyət       0.66      0.84      0.74        50
     Siyasət       0.85      0.85      0.85       234
       İdman       0.92      0.91      0.92       261
İqtisadiyyat       0.93      0.87      0.90       266

    accuracy                           0.85      2000
   macro avg       0.84      0.86      0.84      2000
weighted avg       0.85      0.85      0.85      2000

[[479  69   3  11   3   4]
 [ 58 511  17  11  16   7]
 [  2   6  42   0   0   0]
 [ 13  14   2 198   1   6]
 [  7  15   0   1 238   0]
 [  8  13   0  13   1 231]]
Train Results:
              precision    recall  f1-score   support

       Dünya       0.90      0.87      0.88     13318
     Maraqlı       0.86      0.89      0.87     14684
  Mədəniyyət       0.69      0.93      0.79      1108
     Siyasət       0.89      0.89      0.89     

In [38]:
model2= MultinomialNB()
model2.fit(X_train_vectorized, y_train)
explain_model(model2, X_train_vectorized, X_test_vectorized, y_train, y_test)

Test Results:
              precision    recall  f1-score   support

       Dünya       0.79      0.76      0.77       592
     Maraqlı       0.75      0.73      0.74       649
  Mədəniyyət       0.17      1.00      0.29        11
     Siyasət       0.80      0.77      0.79       242
       İdman       0.88      0.91      0.90       251
İqtisadiyyat       0.86      0.84      0.85       255

    accuracy                           0.78      2000
   macro avg       0.71      0.83      0.72      2000
weighted avg       0.80      0.78      0.79      2000

[[448 107   1  26   6   4]
 [ 88 473  45  10  20  13]
 [  0   0  11   0   0   0]
 [ 15  14   7 187   2  17]
 [  7  14   0   1 229   0]
 [  9  20   0  10   2 214]]
Train Results:
              precision    recall  f1-score   support

       Dünya       0.83      0.77      0.80     13941
     Maraqlı       0.77      0.76      0.76     15416
  Mədəniyyət       0.11      0.99      0.19       160
     Siyasət       0.83      0.82      0.83     

In [39]:
text = '''
Budapeşt, 29 dekabr, AZƏRTAC

Budapeştdə Yeni il və Dünya Azərbaycanlılarının Həmrəyliyi Günü münasibətilə bayram tədbiri keçirilib.

AZƏRTAC xəbər verir ki, Azərbaycan-Macarıstan Məzunlar Birliyinin (AHAU) təşkilatçılığı və Macarıstandakı Azərbaycan Evinin dəstəyi ilə baş tutan tədbirdə Budapeştdə yaşayan azərbaycanlı icma üzvləri bir araya gəliblər.

Tədbir Macarıstandakı Azərbaycan Evinin rəhbəri İbrahim Səfərli və Azərbaycan-Macarıstan Məzunlar Birliyinin həmtəsisçisi Günay Bayramovanın açılış nitqləri ilə başlayıb. Soydaşlarımız Dünya Azərbaycanlılarının Həmrəyliyi Günü və Yeni il münasibətilə təbrik olunub, diaspor fəaliyyətində birliyin, həmrəyliyin və birgə təşəbbüslərin əhəmiyyəti xüsusi vurğulanıb.

Qeyd edilib ki, Ümummilli Lider Heydər Əliyevin təşəbbüsü ilə təsis edilən Dünya Azərbaycanlılarının Həmrəyliyi Günü Azərbaycanın özündə və dünyanın 70-dən çox ölkəsində geniş şəkildə qeyd olunur. Bu əlamətdar gün xaricdə yaşayan həmvətənlərimiz arasında milli birlik və təşkilatlanmanın gücləndirilməsi baxımından mühüm əhəmiyyət daşıyır.

Sonda tədbir iştirakçılar arasında ünsiyyəti gücləndirən şəbəkələşmə oyunları və milli yallı rəqsi ilə xoş və səmimi ab-havada yekunlaşıb.
'''

In [40]:
print(model2.predict(text_pipeline.transform([text])))

['Siyasət']
